In [ ]:
import multiprocessing
import pandas as pd
import numpy as np
import scipy.stats as sc
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.collections import PatchCollection
import dask.dataframe as dd
import os


multiprocessing.cpu_count()

Read in the data and clean it up

In [ ]:
# path to file
home = os.path.expanduser('~')
path = f'{home}/UrbanForest/all_clean_LAcounty_sunset.hdf'

# read the hdf
la = pd.read_hdf(path, key='data')

# select desired columns
cols=['ID', 'LATITUDE', 'LONGITUDE', 'DBH_LO', 'DBH_HI', 'CREATED',
      'UPDATED', 'SOURCE', 'Name_matched', 'Zone']
la = la[cols]

# drop NAs
la.dropna(how='any', axis=0, subset=['DBH_LO', 'DBH_HI'], inplace=True)

# capitalize genus names
la['Name_matched'] = la.Name_matched.str.capitalize()

# convert DBH to cm
la['dbh_low']  = 2.54 * la.DBH_LO
la['dbh_high'] = 2.54 * la.DBH_HI
la.drop(['DBH_LO', 'DBH_HI'], axis=1, inplace=True)

# Change date fields to dateTime type
la['created'] = pd.to_datetime(la.CREATED)
la['updated'] = pd.to_datetime(la.UPDATED)
la.drop(['CREATED', 'UPDATED'], axis=1, inplace=True)


 We will first use allometric equations from :

 McPherson, E. Gregory; van Doorn, Natalie S.; Peper, Paula J. 2016. Urban tree database.
 Fort Collins, CO: Forest Service Research Data Archive. Updated 21 January 2020.
 https://doi.org/10.2737/RDS-2016-0005

 'Apps min' and 'Apps max' give the input range (cm) that the authors feel 
  that the equations are reliable
 'InlEmp' and 'SoCalC' are Climate zones where the eqs are different.
  SoCalC reference city is Santa Monica, InlEmp is Claremont,
  see Table 1, p16 for further Climate zone details.  
  
  After reading the equations and coefficients, we will get rid of trees that only occur a few times, and trees that we o not have equations for.

In [ ]:
# The equations
def mcpherson_eqs():
    '''returns dict of equations from table 3 (p24) of McPherson 2020
    functions use np so as to be vectorized'''

    eq_dict = {'lin'        : (lambda a, b, c, d, e, x, mse: a + b * (x)), 
                'quad'      : (lambda a, b, c, d, e, x, mse: a + b * x + c * x**2),
                'cub'      : (lambda a, b, c, d, e, x, mse: a + b * x + c * x**2 + d * x**3),
                'quart'     : (lambda a, b, c, d, e, x, mse:a + b * x + c *x**2 + d * x**3 + e * x**4), 
                'loglogw1' : (lambda a, b, c, d, e, x, mse: np.exp(a + b * np.log(np.log(x + 1) + (mse/2)))),
                'loglogw2' : (lambda a, b, c, d, e, x, mse: np.exp(a + b * np.log(np.log(x + 1)) + (np.sqrt(x) + (mse/2)))),
                'loglogw3' : (lambda a, b, c, d, e, x, mse: np.exp(a + b * np.log(np.log(x + 1)) + (x) + (mse/2))),
                'loglogw4' : (lambda a, b, c, d, e, x, mse: np.exp(a + b * np.log(np.log(x + 1)) + (x**2) + (mse/2))),
                'expow1'    : (lambda a, b, c, d, e, x, mse: np.exp(a+ b * (x) + (mse/2))),
                'expow2'    : (lambda a, b, c, d, e, x, mse: np.exp(a + b * (x) + np.sqrt(x) + (mse/2))),
                'expow3'    : (lambda a, b, c, d, e, x, mse: np.exp(a + b * (x) + (x) + (mse/2))),
                'expow4'    : (lambda a, b, c, d, e, x, mse: np.exp(a + b * (x) + (x**2) + (mse/2)))}

    return(eq_dict)

eq_dict = mcpherson_eqs()

# The cooeficients
coef_df = pd.read_csv('TS6_Growth_coefficients.csvx',
usecols=['Region', 'Scientific Name', 'Independent variable', 'Predicts component ', 'EqName', 'Units of predicted components',
'EqName', 'a', 'b', 'c', 'd', 'e', 'Apps min', 'Apps max'])

# Find all the trees with over 100 occurances in the dataset
trees = la.Name_matched.value_counts()
trees = list(trees.where(trees > 100).dropna().index)

# drop trees we do not have equations for
trees = [s for s in trees if s in coef_df['Scientific Name'].unique()]
la = la.loc[la.Name_matched.isin(trees)]

la.info()

The USGS lidar data is hosted on amazon, so we will nned the AWS client to access it.

In [ ]:
#!curl "https://s3.amazonaws.com/aws-cli/awscli-bundle.zip" -o "awscli-bundle.zip"
#!unzip awscli-bundle.zip 
#!./awscli-bundle/install -b ~/bin/aws

Let's make a tmp directory too

In [ ]:
# make a tmp directory

!mkdir ~/tmp

# make a variable for its path
tmp = f'{home}/tmp'

# make a variable with the path to aws cli
aws = '~/bin/aws'

Download the top level ept json for the ```USGS_LPC_CA_LosAngeles_2016_LAS_2018``` dataset.

In [ ]:
import subprocess
cmd = f'{aws} s3 cp s3://usgs-lidar-public/USGS_LPC_CA_LosAngeles_2016_LAS_2018/ept.json {tmp} --no-sign-request'
subprocess.run(cmd, shell=True, capture_output=True)

We will load ```ept.json``` and extract usefull information

In [ ]:
import json
with open(f'{tmp}/ept.json') as f:
    meta = json.load(f)
    
bounds = meta['bounds']
bounds_conf = meta['boundsConforming']
srs    = meta['srs']
span   = meta['span']
schema  = meta['schema']
srs

The above output tells us the data is in EPSG:3857.  There is only a horizontal code present.  Lets reduce the srs to a more sueful form for later

In [ ]:
srs = meta['srs']['authority'] + ':' + meta['srs']['horizontal']

Because the srs is EPSG:3857 (Pseudo-Mercator) I don't think it necesaary to reproject.  We do need to use the scale and offset values to find absolute position according to ``read_value * scale + offset```

In [ ]:
def bag_scale_offset(name, schema):
    '''Retruns scale and offset for the spatial dimension given by name'''
    for thing in schema:
        if thing['name'] == name:
            return(thing['scale'], thing['offset'])
        
x_scale, x_offset = bag_scale_offset('X', schema)
y_scale, y_offset = bag_scale_offset('Y', schema)
z_scale, z_offset = bag_scale_offset('Z', schema)

def rescale(lon, lat, elev=None):
    '''Returns point rescaled to the ept coords'''
    x = lon * x_scale + x_offset
    y = lat * y_scale + y_offset
    if elev:
        z = elev * z_scale + z_offset
        return(x, y, z)
    return(x, y)



Now it should be possible to define a bounding box around a tree to query the pointcloud at that local.

In [ ]:
# For now we will ad 0.00007 degrees in each direction, this is jus a guess based on 5th decimal place ~ 1.1m
# also not setting z max and min for the moment
def make_scaled_bbox(lat, lon, bounds=None):
    '''Returns a bbox in ept coords.
    If present bounds is of form [xmin, ymin, zmin, xmax, ymax, zmax]'''
    
    buf = 0.00007
    xmin = lon - buf
    ymin = lat - buf
    xmax = lon + buf
    ymax = lat + buf
    xmin, ymin = rescale(xmin, ymin)
    xmax, ymax = rescale(xmax, ymax)

    # make sure no bbox is out of the ept bbox
    if bounds:
        xmin = max(xmin, bounds[0])
        ymin = max(ymin, bounds[1])
        xmax = min(xmax, bounds[3])
        ymax = min(ymax, bounds[4])
    
    return([xmin, xmax], [ymin, ymax])

def make_bbox(lat, lon):
    buf = 0.00007
    xmin = lon - buf
    ymin = lat - buf
    xmax = lon + buf
    ymax = lat + buf
    return([xmin, xmax], [ymin, ymax])

    

In [ ]:
#pip install pdal

Now lets try to get the point cloud within the bbox using PDALs ept reader

In [ ]:
import pdal
from string import Template
from tqdm import tqdm
from dask import delayed
from dask import compute
from dask.diagnostics import ProgressBar

In [ ]:
id = 1986759
lat = la.loc[la.ID==id]['LATITUDE'].values[0]
lon = la.loc[la.ID==id]['LONGITUDE'].values[0]
lon

In [ ]:
with open(f'{tmp}/ept.json') as f:
    meta = json.load(f)

scaled_bbox = make_scaled_bbox(lat, lon, bounds=bounds_conf)

def bbox_geojson(lat, lon, filename):
    '''makes wgs84 bbox as geojson for comparison in gis'''
    [xmin, xmax], [ymin, ymax] = make_bbox(lat, lon)
    gjson = {'coordinates' : [[[xmin, ymin], [xmin, ymax], [xmax, ymax], [xmax, ymin]]],
            'type' : 'Polygon'}
    with open(filename, 'w') as of:
        json.dump(gjson, of)

bbox_geojson(lat, lon, 'xxx.json')        
print('geojson written ...')

# make and validate pipeline
t = Template('''
{
    "pipeline": [
        {
        "bounds": "([-10425171.940, -10425171.000], [5164494.710, 5164595.200])",
        "filename": "https://s3-us-west-2.amazonaws.com/usgs-lidar-public/USGS_LPC_CA_LosAngeles_2016_LAS_2018/ept.json",
        "type": "readers.ept",
        "tag": "readdata"
        },
        {
        "out_srs": "EPSG:4326",
        "type": "filters.reprojection"
        },
        {
        "filename": "xxx.laz",
        "type": "writers.las"
        },
        {
        "filename": "xxx.tif",
        "gdalopts": "tiled=yes,     compress=deflate",
        "nodata": -9999,
        "output_type": "idw",
        "resolution": 1,
        "type": "writers.gdal",
        "window_size": 6
        }
    ]
}''')

pipe = t.substitute(scaled_bbox=scaled_bbox)
print('pipeline json written ...')
pipeline = pdal.Pipeline(pipe)
print('pipeline string turned into pipeline ...')
pipeline.validate()
print('pipeline validated ...')

 # execuite pipeline
count = pipeline.execute()
print('pipeline executed ...')
S = pipeline.arrays[0]
metadata = pipeline.metadata
log = pipeline.log

print(S.shape)

In [ ]:
scaled_bbox

In [ ]:
meta

In [15]:
@delayed
def row_bounds_ept_query(i):
    # get lat lon of first entry
    row = la.iloc[i]
    ident = row['ID']
    lat = row['LATITUDE']
    lon = row['LONGITUDE']

    # make bbox in the ept coord system
    scaled_bbox = make_scaled_bbox(lat, lon, bounds=bounds_conf)
    
    # sanity check will raise erros if scalled bbox is not in the ept bounds
    assert (scaled_bbox[0][0] > bounds[0]) & (scaled_bbox[0][1] < bounds[3])
    assert (scaled_bbox[1][0] > bounds[1]) & (scaled_bbox[1][1] < bounds[4])
    
    # make and validate pipeline
    t = Template('''
    [
        {
            "type": "readers.ept",
            "filename": "https://s3-us-west-2.amazonaws.com/usgs-lidar-public/USGS_LPC_CA_LosAngeles_2016_LAS_2018/ept.json",
            "bounds": "${scaled_bbox}",
            "resolution": 1
        }
    ]''')

    pipe = t.substitute(scaled_bbox=tbbox)
    pipeline = pdal.Pipeline(pipe)
    pipeline.validate()
    
    # execuite pipeline
    count = pipeline.execute()
    S = pipeline.arrays[0]
    metadata = pipeline.metadata
    log = pipeline.log
    
    # do stuff
    sh = S.shape
    if sh[0] > 0:
        print(f'{ident} is a numpy structured array of shape {sh}.')
        return(S)
        
results = []
for i in range(1):
    results.append(row_bounds_ept_query(i+10))
    
with ProgressBar():
    S = compute(*results)

IndentationError: unexpected indent (<ipython-input-15-5c063dddf927>, line 18)